<a href="https://colab.research.google.com/github/igorrivin/svm-pytorch/blob/master/convnet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
LOG_DIR = './my_graph/mnist'
get_ipython().system_raw(
    'tensorboard --logdir {} --host 0.0.0.0 --port 6006 &'
    .format(LOG_DIR)
)

In [0]:
get_ipython().system_raw('./ngrok http 6006 &')

In [3]:
!curl -s http://localhost:4040/api/tunnels | python3 -c \
    "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"

https://c3d39de5.ngrok.io


In [4]:
""" Using convolutional net on MNIST dataset of handwritten digit
(http://yann.lecun.com/exdb/mnist/)
"""
from __future__ import print_function
from __future__ import division
from __future__ import print_function

import os

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

import time
import utils

import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data


N_CLASSES = 10

# Step 1: Read in data
# using TF Learn's built in function to load MNIST data to the folder data/mnist
mnist = input_data.read_data_sets("/data/mnist", one_hot=True)

# Step 2: Define paramaters for the model
LEARNING_RATE = 0.001
BATCH_SIZE = 128
SKIP_STEP = 10
DROPOUT = 0.75
N_EPOCHS = 1

# Step 3: create placeholders for features and labels
# each image in the MNIST data is of shape 28*28 = 784
# therefore, each image is represented with a 1x784 tensor
# We'll be doing dropout for hidden layer so we'll need a placeholder
# for the dropout probability too
# Use None for shape so we can change the batch_size once we've built the graph
with tf.name_scope('data'):
    X = tf.placeholder(tf.float32, [None, 784], name="X_placeholder")
    Y = tf.placeholder(tf.float32, [None, 10], name="Y_placeholder")

dropout = tf.placeholder(tf.float32, name='dropout')

# Step 4 + 5: create weights + do inference
# the model is conv -> relu -> pool -> conv -> relu -> pool -> fully connected -> softmax

global_step = tf.Variable(0, dtype=tf.int32, trainable=False, name='global_step')

import pathlib
pathlib.Path('checkpoints').mkdir(parents=True, exist_ok=True) 
pathlib.Path('checkpoints/convnet_mnist').mkdir(parents=True, exist_ok=True) 
#utils.make_dir('checkpoints')
#utils.make_dir('checkpoints/convnet_mnist')


Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting /data/mnist/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting /data/mnist/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting /data/mnist/t10k-images-idx3-ubyte.gz
Extracting /data/mnist/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [5]:
!pip install utils

In [0]:
with tf.variable_scope('conv1') as scope:
    # first, reshape the image to [BATCH_SIZE, 28, 28, 1] to make it work with tf.nn.conv2d
    # use the dynamic dimension -1
    images = tf.reshape(X, shape=[-1, 28, 28, 1])

    # TO DO
    size_in = 1
    size_out =32
    # create kernel variable of dimension [5, 5, 1, 32]
    # use tf.truncated_normal_initializer()
    w = tf.get_variable('W',shape=[5, 5, size_in, size_out], initializer = tf.truncated_normal_initializer())
    

    # TO DO

    # create biases variable of dimension [32]
    # use tf.constant_initializer(0.0)
    #b = tf.Variable(tf.constant(0.1, shape=[size_out]), name="B")
    b = tf.get_variable("B",shape=[size_out], dtype=tf.float32, trainable = True, initializer = tf.constant_initializer(0.0))
    # TO DO

    # apply tf.nn.conv2d. strides [1, 1, 1, 1], padding is 'SAME'
    conv = tf.nn.conv2d(images, w, strides=[1, 1, 1, 1], padding="SAME")
    # TO DO
   
    # apply relu on the sum of convolution output and biases

    # TO DO
    conv1 = tf.nn.relu(conv + b, name=scope.name)
    tf.summary.histogram("weights", w)
    tf.summary.histogram("biases", b)
    tf.summary.histogram("activations", conv1)
    # output is of dimension BATCH_SIZE x 28 x 28 x 32


In [0]:
with tf.variable_scope('pool1') as scope:
  # apply max pool with ksize [1, 2, 2, 1], and strides [1, 2, 2, 1], padding 'SAME'

  # TO DO
  pool1 = tf.nn.max_pool(conv1, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding="SAME", name=scope.name)
  # output is of dimension BATCH_SIZE x 14 x 14 x 32

In [0]:
with tf.variable_scope('conv2') as scope:
    # similar to conv1, except kernel now is of the size 5 x 5 x 32 x 64
    kernel = tf.get_variable('kernels', [5, 5, 32, 64],
                             initializer=tf.truncated_normal_initializer())
    biases = tf.get_variable('biases', [64],
                             initializer=tf.random_normal_initializer())
    conv = tf.nn.conv2d(pool1, kernel, strides=[1, 1, 1, 1], padding='SAME')
    conv2 = tf.nn.relu(conv + biases, name=scope.name)

    # output is of dimension BATCH_SIZE x 14 x 14 x 64

In [0]:
with tf.variable_scope('pool2') as scope:
    # similar to pool1
    pool2 = tf.nn.max_pool(conv2, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1],
                           padding='SAME', name=scope.name)

    # output is of dimension BATCH_SIZE x 7 x 7 x 64

In [0]:
with tf.variable_scope('fc') as scope:
    # use weight of dimension 7 * 7 * 64 x 1024
    input_features = 7 * 7 * 64
    size_out=1024

    # create weights and biases
    w = tf.Variable(tf.truncated_normal([input_features, size_out], stddev=0.1), name="W")
    b = tf.Variable(tf.constant(0.1, shape=[size_out]), name="B")
    # TO DO

    # reshape pool2 to 2 dimensional
    pool2 = tf.reshape(pool2, [-1, input_features])

    # apply relu on matmul of pool2 and w + b
    fc = tf.nn.relu(tf.matmul(pool2, w) + b, name='relu')

    # TO DO
    tf.summary.histogram("weights", w)
    tf.summary.histogram("biases", b)
    tf.summary.histogram("activations", fc)
    # apply dropout
    fc = tf.nn.dropout(fc, dropout, name='relu_dropout')
    


In [0]:
with tf.variable_scope('softmax_linear') as scope:
  # this you should know. get logits without softmax
  # you need to create weights and biases
  w = tf.Variable(tf.truncated_normal([1024, 10], stddev=0.1), name="W")
  b = tf.Variable(tf.constant(0.1, shape=[10]), name="B")
  logits = fc@w + b
  # TO DO

In [12]:
# Step 6: define loss function
# use softmax cross entropy with logits as the loss function
# compute mean cross entropy, softmax is applied internally
with tf.name_scope('loss'):
  # you should know how to do this too
  loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=Y, logits=logits), name=scope.name)
  tf.summary.scalar("loss", loss)

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.



In [0]:
optimizer = tf.train.GradientDescentOptimizer(LEARNING_RATE).minimize(loss, global_step=global_step)

In [14]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    saver = tf.train.Saver()
    # to visualize using TensorBoard
    writer = tf.summary.FileWriter('./my_graph/mnist')
    writer.add_graph(sess.graph)
    ##### You have to create folders to store checkpoints
    ckpt = tf.train.get_checkpoint_state(os.path.dirname('checkpoints/convnet_mnist/checkpoint'))
    # if that checkpoint exists, restore from checkpoint
    if ckpt and ckpt.model_checkpoint_path:
        saver.restore(sess, ckpt.model_checkpoint_path)
    summ = tf.summary.merge_all()
    sess.run(global_step.initializer)
    initial_step = global_step.eval()

    start_time = time.time()
    n_batches = int(mnist.train.num_examples / BATCH_SIZE)

    total_loss = 0.0
    for index in range(initial_step, n_batches * N_EPOCHS):  # train the model n_epochs times
        X_batch, Y_batch = mnist.train.next_batch(BATCH_SIZE)
        _, loss_batch = sess.run([optimizer, loss],
                                 feed_dict={X: X_batch, Y: Y_batch, dropout: DROPOUT})
        total_loss += loss_batch
        if (index + 1) % SKIP_STEP == 0:
            print('Average loss at step {}: {:5.1f}'.format(index + 1, total_loss / SKIP_STEP))
            total_loss = 0.0
            saver.save(sess, 'checkpoints/convnet_mnist/mnist-convnet', index)

    print("Optimization Finished!")  # should be around 0.35 after 25 epochs
    print("Total time: {0} seconds".format(time.time() - start_time))

    # test the model
    n_batches = int(mnist.test.num_examples / BATCH_SIZE)
    total_correct_preds = 0
    for i in range(n_batches):
        X_batch, Y_batch = mnist.test.next_batch(BATCH_SIZE)
        _, loss_batch, logits_batch = sess.run([optimizer, loss, logits],
                                               feed_dict={X: X_batch, Y: Y_batch, dropout: DROPOUT})
        s = sess.run(summ,feed_dict={X: X_batch, Y: Y_batch, dropout: DROPOUT})
        writer.add_summary(s, i)
        preds = tf.nn.softmax(logits_batch)
        correct_preds = tf.equal(tf.argmax(preds, 1), tf.argmax(Y_batch, 1))
        accuracy = tf.reduce_sum(tf.cast(correct_preds, tf.float32))
        total_correct_preds += sess.run(accuracy)

    print("Accuracy {0}".format(total_correct_preds / mnist.test.num_examples))

INFO:tensorflow:Restoring parameters from checkpoints/convnet_mnist/mnist-convnet-419
Average loss at step 10:   1.0
Average loss at step 20:   0.9
Average loss at step 30:   0.9
Average loss at step 40:   0.7
Average loss at step 50:   1.0
Average loss at step 60:   1.1
Average loss at step 70:   0.9
Average loss at step 80:   0.8
Average loss at step 90:   1.0
Average loss at step 100:   1.2
Average loss at step 110:   1.2
Average loss at step 120:   0.9
Average loss at step 130:   0.9
Average loss at step 140:   1.1
Average loss at step 150:   0.9
Average loss at step 160:   1.0
Average loss at step 170:   0.8
Average loss at step 180:   1.2
Average loss at step 190:   0.9
Average loss at step 200:   0.9
Average loss at step 210:   1.2
Average loss at step 220:   0.9
Average loss at step 230:   0.8
Average loss at step 240:   0.9
Average loss at step 250:   1.0
Average loss at step 260:   0.9
Average loss at step 270:   0.9
Average loss at step 280:   1.0
Average loss at step 290:  